In [140]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

In [141]:
# load the dataset
data=pd.read_csv('churn_modelling.csv')
data.head()

RowNumber  CustomerId   Surname  CreditScore Geography  Gender  Age  \
0          1    15634602  Hargrave          619    France  Female   42   
1          2    15647311      Hill          608     Spain  Female   41   
2          3    15619304      Onio          502    France  Female   42   
3          4    15701354      Boni          699    France  Female   39   
4          5    15737888  Mitchell          850     Spain  Female   43   

   Tenure    Balance  NumOfProducts  HasCrCard  IsActiveMember  \
0       2       0.00              1          1               1   
1       1   83807.86              1          0               1   
2       8  159660.80              3          1               0   
3       1       0.00              2          0               0   
4       2  125510.82              1          1               1   

   EstimatedSalary  Exited  
0        101348.88       1  
1        112542.58       0  
2        113931.57       1  
3         93826.63       0  
4         79084.10       0

In [142]:
# Preprocess the data
# drop irrelevant columns
data = pd.read_csv('churn_modelling.csv')  # Reload the original data
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)
data


CreditScore Geography  Gender  Age  Tenure    Balance  NumOfProducts  \
0             619    France  Female   42       2       0.00              1   
1             608     Spain  Female   41       1   83807.86              1   
2             502    France  Female   42       8  159660.80              3   
3             699    France  Female   39       1       0.00              2   
4             850     Spain  Female   43       2  125510.82              1   
...           ...       ...     ...  ...     ...        ...            ...   
9995          771    France    Male   39       5       0.00              2   
9996          516    France    Male   35      10   57369.61              1   
9997          709    France  Female   36       7       0.00              1   
9998          772   Germany    Male   42       3   75075.31              2   
9999          792    France  Female   28       4  130142.79              1   

      HasCrCard  IsActiveMember  EstimatedSalary  Exited  
0             1               1        101348.88       1  
1             0               1        112542.58       0  
2             1               0        113931.57       1  
3             0               0         93826.63       0  
4             1               1         79084.10       0  
...         ...             ...              ...     ...  
9995          1               0         96270.64       0  
9996          1               1        101699.77       0  
9997          0               1         42085.58       1  
9998          1               0         92888.52       1  
9999          1               0         38190.78       0  

[10000 rows x 11 columns]

In [143]:
# Encode categorical variables
label_encoder_gender=LabelEncoder()
data['Gender']=label_encoder_gender.fit_transform(data['Gender'])
data

CreditScore Geography  Gender  Age  Tenure    Balance  NumOfProducts  \
0             619    France       0   42       2       0.00              1   
1             608     Spain       0   41       1   83807.86              1   
2             502    France       0   42       8  159660.80              3   
3             699    France       0   39       1       0.00              2   
4             850     Spain       0   43       2  125510.82              1   
...           ...       ...     ...  ...     ...        ...            ...   
9995          771    France       1   39       5       0.00              2   
9996          516    France       1   35      10   57369.61              1   
9997          709    France       0   36       7       0.00              1   
9998          772   Germany       1   42       3   75075.31              2   
9999          792    France       0   28       4  130142.79              1   

      HasCrCard  IsActiveMember  EstimatedSalary  Exited  
0             1               1        101348.88       1  
1             0               1        112542.58       0  
2             1               0        113931.57       1  
3             0               0         93826.63       0  
4             1               1         79084.10       0  
...         ...             ...              ...     ...  
9995          1               0         96270.64       0  
9996          1               1        101699.77       0  
9997          0               1         42085.58       1  
9998          1               0         92888.52       1  
9999          1               0         38190.78       0  

[10000 rows x 11 columns]

In [144]:
# onehot encode the geography column
from sklearn.preprocessing import OneHotEncoder
onehot_encoder_geo=OneHotEncoder()
geo_encoder=onehot_encoder_geo.fit_transform(data[['Geography']])
geo_encoder


<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [145]:
onehot_encoder_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [146]:
geo_encoded_df= pd.DataFrame(geo_encoder.toarray(),columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

Geography_France  Geography_Germany  Geography_Spain
0                  1.0                0.0              0.0
1                  0.0                0.0              1.0
2                  1.0                0.0              0.0
3                  1.0                0.0              0.0
4                  0.0                0.0              1.0
...                ...                ...              ...
9995               1.0                0.0              0.0
9996               1.0                0.0              0.0
9997               1.0                0.0              0.0
9998               0.0                1.0              0.0
9999               1.0                0.0              0.0

[10000 rows x 3 columns]

In [147]:
# Combine one hot encoder column with the original data
data = pd.concat([data, geo_encoded_df], axis=1)
data.head()


CreditScore Geography  Gender  Age  Tenure    Balance  NumOfProducts  \
0          619    France       0   42       2       0.00              1   
1          608     Spain       0   41       1   83807.86              1   
2          502    France       0   42       8  159660.80              3   
3          699    France       0   39       1       0.00              2   
4          850     Spain       0   43       2  125510.82              1   

   HasCrCard  IsActiveMember  EstimatedSalary  Exited  Geography_France  \
0          1               1        101348.88       1               1.0   
1          0               1        112542.58       0               0.0   
2          1               0        113931.57       1               1.0   
3          0               0         93826.63       0               1.0   
4          1               1         79084.10       0               0.0   

   Geography_Germany  Geography_Spain  
0                0.0              0.0  
1                0.0              1.0  
2                0.0              0.0  
3                0.0              0.0  
4                0.0              1.0

In [148]:
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)

with open('onehot_encoder_geo.pkl','wb') as file:
    pickle.dump(onehot_encoder_geo,file)



In [149]:

# divide the dataset in to independent and dependent variables
X=data.drop('Exited',axis=1)
y=data['Exited']
X = X.select_dtypes(include=['float64', 'int64'])


# split the dataset in to training and testing sets
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

## Scale these features
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train.as_matrix())
X_test=scaler.transform(X_test.as_matrix())


In [150]:
X_train

array([[ 0.35649971, -0.6557859 ,  0.34567966, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.20389777,  0.29493847, -0.3483691 , ..., -0.99850112,
         1.72572313, -0.57638802],
       [-0.96147213, -1.41636539, -0.69539349, ..., -0.99850112,
        -0.57946723,  1.73494238],
       ...,
       [ 0.86500853, -0.08535128, -1.38944225, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.15932282,  0.3900109 ,  1.03972843, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.47065475,  1.15059039, -1.38944225, ..., -0.99850112,
         1.72572313, -0.57638802]])

In [151]:
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

In [152]:
data

CreditScore Geography  Gender  Age  Tenure    Balance  NumOfProducts  \
0             619    France       0   42       2       0.00              1   
1             608     Spain       0   41       1   83807.86              1   
2             502    France       0   42       8  159660.80              3   
3             699    France       0   39       1       0.00              2   
4             850     Spain       0   43       2  125510.82              1   
...           ...       ...     ...  ...     ...        ...            ...   
9995          771    France       1   39       5       0.00              2   
9996          516    France       1   35      10   57369.61              1   
9997          709    France       0   36       7       0.00              1   
9998          772   Germany       1   42       3   75075.31              2   
9999          792    France       0   28       4  130142.79              1   

      HasCrCard  IsActiveMember  EstimatedSalary  Exited  Geography_France  \
0             1               1        101348.88       1               1.0   
1             0               1        112542.58       0               0.0   
2             1               0        113931.57       1               1.0   
3             0               0         93826.63       0               1.0   
4             1               1         79084.10       0               0.0   
...         ...             ...              ...     ...               ...   
9995          1               0         96270.64       0               1.0   
9996          1               1        101699.77       0               1.0   
9997          0               1         42085.58       1               1.0   
9998          1               0         92888.52       1               0.0   
9999          1               0         38190.78       0               1.0   

      Geography_Germany  Geography_Spain  
0                   0.0              0.0  
1                   0.0              1.0  
2                   0.0              0.0  
3                   0.0              0.0  
4                   0.0              1.0  
...                 ...              ...  
9995                0.0              0.0  
9996                0.0              0.0  
9997                0.0              0.0  
9998                1.0              0.0  
9999                0.0              0.0  

[10000 rows x 14 columns]

In [153]:
X_train.shape[1]

11

### ANN Impementation

In [154]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [155]:
# bulild our ANN model
model=Sequential([
    Dense(units=64,activation='relu',input_shape=(X_train.shape[1],)), # HL1 connected to input layer
    Dense(units=32,activation='relu'), # HL2
    Dense(units=1,activation='sigmoid') # output layer
]
)

In [156]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 64)                768       
                                                                 
 dense_10 (Dense)            (None, 32)                2080      
                                                                 
 dense_11 (Dense)            (None, 1)                 33        
                                                                 
Total params: 2881 (11.25 KB)
Trainable params: 2881 (11.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [157]:
import tensorflow
opt=tensorflow.keras.optimizers.Adam(learning_rate=0.01)
loss=tensorflow.keras.losses.BinaryCrossentropy()
loss

In [158]:
# compile the model for forward and backward propagation
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [159]:
## setup the tensorboard
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [160]:
## Setup the early stopping
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)


In [161]:
## Train the model
history=model.fit(
    X_train,y_train,validation_split=0.2,epochs=100,batch_size=32,callbacks=[tensorflow_callback,early_stopping_callback]
)

Epoch 1/100
200/200 [==============================] - 3s 4ms/step - loss: 0.4733 - accuracy: 0.7970 - val_loss: 0.4107 - val_accuracy: 0.8281
Epoch 2/100
200/200 [==============================] - 1s 4ms/step - loss: 0.4008 - accuracy: 0.8364 - val_loss: 0.3783 - val_accuracy: 0.8481
Epoch 3/100
200/200 [==============================] - 1s 3ms/step - loss: 0.3675 - accuracy: 0.8542 - val_loss: 0.3643 - val_accuracy: 0.8487
Epoch 4/100
200/200 [==============================] - 1s 3ms/step - loss: 0.3529 - accuracy: 0.8566 - val_loss: 0.3578 - val_accuracy: 0.8519
Epoch 5/100
200/200 [==============================] - 1s 4ms/step - loss: 0.3478 - accuracy: 0.8614 - val_loss: 0.3540 - val_accuracy: 0.8537
Epoch 6/100
200/200 [==============================] - 1s 4ms/step - loss: 0.3435 - accuracy: 0.8584 - val_loss: 0.3510 - val_accuracy: 0.8525
Epoch 7/100
200/200 [==============================] - 1s 3ms/step - loss: 0.3424 - accuracy: 0.8603 - val_loss: 0.3487 - val_accuracy: 0.8544

In [162]:
model.save('model.h5')

c:\ANNClassification\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [163]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [164]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 11572), started 22:02:05 ago. (Use '!kill 11572' to kill it.)